# RL Final Project

Now it's finally time to put into use what we have learned so far in this course!

The aim of this project is to assess your practical knowledge in Reinforcement Learning.

your project consist of 2 parts. you will get the chance to work with 2 different environment.


## 2.Atari Game Pong


<img src="zzzzzzzzzzzzzzzzzc"/>

**[Pong](https://www.gymlibrary.dev/environments/atari/pong/)** is a famus atari game that almost all of us have played it at least once!
The goal of this task is to get engage with **gym** library and use Deep Reinforcement Learning to train an agent which can actually play this game!

In [ ]:
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

!pip install ALE gym
!pip install 'gym[accept-rom-license, atari]'
!pip install opencv-python

!pip install 'tensorflow<2.11'
!pip install tensorflow-gpu==2.10

!pip install tqdm
!pip install jdc

Imports the necessary libraries and modules for the code, including Gym (for the RL environment), OpenCV (for image processing), NumPy (for numerical operations), TensorFlow (for deep learning), and Keras (for building and training the DQN model).

In [2]:
import gym
import cv2
import numpy as np
import tensorflow as tf
import warnings
import random
import jdc

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam
from IPython.utils import io
from tqdm.notebook import tqdm

Define important constant

In [3]:
INPUT_SHAPE = (84, 84, 1)
TRAIN = True
GAME_NAME = 'ALE/Pong-v5'

Supress unimportant warnings

In [4]:
warnings.filterwarnings('ignore')

Sets up the TensorFlow session to run on the GPU. It configures the session with GPU options and sets it as the backend for Keras.

In [5]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

Defines the preprocess_frame function, which takes an observation from the environment and preprocesses the frame. It converts the frame to grayscale, resizes it to (84, 84) pixels, and normalizes the pixel values to the range [0, 1].

In [6]:
def preprocess_frame(frame):
    frame = frame[0]
    if frame.ndim > 2:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    return cv2.resize(frame, (84, 84)) / 255.0

Defines the DQNAgent class, which represents the DQN (Deep Q-Network) agent. The DQNAgent has methods to build the model, get an action, store experiences, replay experiences for training, and run episodes. The model is built using the Keras Sequential API and consists of convolutional layers, fully connected layers, and an output layer. The model is compiled with the Adam optimizer and mean squared error (MSE) loss.

In [7]:
class DQNAgent:
    def __init__(self, learning_rate=0.00025, batch_size=32, memory_size=10000):
        self.env = gym.make(GAME_NAME)
        self.action_space = self.env.action_space.n
        self.model = self.build_model(learning_rate)
        self.batch_size = batch_size
        self.memory = []
        self.memory_size = memory_size

    def build_model(self, learning_rate, activation='relu'):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(8, 8), strides=4, activation=activation, input_shape=INPUT_SHAPE))
        model.add(Conv2D(64, kernel_size=(4, 4), strides=2, activation=activation))
        model.add(Conv2D(64, kernel_size=(3, 3), strides=1, activation=activation))
        model.add(Flatten())
        model.add(Dense(512, activation=activation))
        model.add(Dense(self.action_space))

        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='mse')
        return model

Adds a method get_action to the DQNAgent class. It takes a state and an epsilon value for epsilon-greedy exploration. With probability epsilon, it selects a random action. Otherwise, it uses the model to predict the Q-values for the given state and selects the action with the highest Q-value.

In [8]:
%%add_to DQNAgent

def get_action(self, state, epsilon):
    if np.random.rand() <= epsilon:
        return np.random.randint(self.action_space)
    with io.capture_output() as captured:
        q_values = self.model.predict(state)
    return np.argmax(q_values[0])

Adds a method remember to the DQNAgent class. It stores the agent's experience tuple (state, action, reward, next_state, done) in the memory. If the memory exceeds the specified memory size, the oldest experiences are removed to maintain the memory size.

In [9]:
%%add_to DQNAgent

def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))
    if len(self.memory) > self.memory_size:
        self.memory = self.memory[-self.memory_size:]

Adds a method replay to the DQNAgent class. It performs the model training using the experience replay technique. It samples a batch of experiences from the memory and calculates the target Q-values using the Bellman equation. The model is then updated using the states and target Q-values.

In [10]:
%%add_to DQNAgent

def replay(self, gamma):
    if len(self.memory) < self.batch_size:
        return
    batch = random.sample(self.memory, self.batch_size)
    states = []
    targets = []
    for state, action, reward, next_state, done in batch:
        target = reward
        if not done:
            q_values = self.model.predict(next_state)
            target += gamma * np.max(q_values[0])
        target_f = self.model.predict(state)
        target_f[0][action] = target
        states.append(state)
        targets.append(target_f)

    states = np.array([state[0] for state in states])
    targets = np.array(targets)

    self.model.fit(states, targets, batch_size=self.batch_size, verbose=0)

Adds a method train to the DQNAgent class. It trains the DQN agent for a specified number of episodes. It uses a progress bar to track the progress of the training. In each episode, it calls the run_episode method to run a single episode and update the model's weights. The epsilon value is decayed over episodes to gradually shift from exploration to exploitation. After training, the model is saved to a file.

In [11]:
%%add_to DQNAgent

def train(self, model_name, episodes, epsilon_decay, epsilon_start=1.0, epsilon_end=0.1, gamma=0.97,
          max_episode_length=100):
    epsilon = epsilon_start
    bar_format = 'Training: {percentage:3.0f}% |{bar}| Elapsed: {elapsed}{postfix}'
    training_pbar = tqdm(total=episodes, bar_format=bar_format, unit='episode')

    for episode in range(episodes):
        total_reward = self.run_episode(epsilon, gamma, max_episode_length)
        training_pbar.set_postfix_str(f'Reward: {total_reward}')
        training_pbar.update(1)
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        with io.capture_output() as captured:
            self.replay(gamma)

    training_pbar.close()
    print('Training completed.')

    self.model.save(model_name)
    print('Model saved.')

    self.env.close()

Adds a method run_episode to the DQNAgent class. It runs a single episode of the environment using the current policy and updates the model's weights. It iteratively selects actions, observes the next state and reward, and performs a model update using the Q-learning algorithm. The progress is tracked using a separate progress bar.

In [12]:
%%add_to DQNAgent

def run_episode(self, epsilon, gamma, max_episode_length):
    observation = self.env.reset()
    state = preprocess_frame(observation)
    state = np.reshape(state, (1, *INPUT_SHAPE))
    done = False
    total_reward = 0
    episode_length = 0

    # bar_format = 'Episode: {percentage:3.0f}% |{bar}| Speed: {rate_fmt}{postfix}'
    # episode_pbar = tqdm(total=max_episode_length, bar_format=bar_format, unit='step')

    while not done:
        action = self.get_action(state, epsilon)
        next_observation, reward, done, _, _ = self.env.step(action)
        next_state = preprocess_frame(next_observation)
        if next_state is not None:
            next_state = np.reshape(next_state, (1, *INPUT_SHAPE))
            with io.capture_output() as captured:
                self.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            episode_length += 1
            # episode_pbar.set_postfix_str(f'Reward: {total_reward}')
            # episode_pbar.update(1)

        if episode_length >= max_episode_length:
            done = True

    # episode_pbar.close()
    return total_reward

Creates an instance of the DQNAgent class. It specifies the input shape, action space size, and learning rate for the agent. Trains the agent by calling the train method. It specifies the model name for saving, the number of episodes, epsilon decay rate, and maximum episode length. After training, it closes the environment.

In [13]:
if TRAIN:
    agent = DQNAgent()

    agent.train(
        model_name='trained_model.h5',
        episodes=100,
        epsilon_decay=0.99,
        max_episode_length=1000
    )

Training:   0% |          | Elapsed: 00:00, Remaining: ?

Training completed.
Model saved.


To open and play with the trained models

In [19]:
def open_model(model_name):
    model = load_model(model_name)
    env = gym.make(GAME_NAME, render_mode='human')
    observation = env.reset()
    done = False
    observation = env.reset()
    state = preprocess_frame(observation)
    state = np.reshape(state, (1, *INPUT_SHAPE))

    while not done:
        env.render()
        action = np.argmax(model.predict(state))
        next_observation, reward, done, _, _ = env.step(action)
        next_state = preprocess_frame(next_observation)
        next_state = np.reshape(next_state, (1, *INPUT_SHAPE))
        state = next_state


open_model('trained_model.h5')

1/1 [==============================] - 0s 22ms/step


KeyboardInterrupt: 

**Note**: Keep in mind that observation space for this environment are frames from environment. Observation space is an image of size (210, 160, 3). so you will need to implement an agent which can process images!(a CNN based agent). 

Make sure to do perform preprocessing on the frames. For example, you can convert the RBG image to gray. you can use [OpenCV](https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html) library to perform resize\ing, bluring or any applicable filtering on the frames.

## Grading criteria
Project: 35 points

* Final Viva: 10 points
* Implementation: 10 points
* Final Report: 15 points

For viva you will need to expilictly mention each team member's contribution.

You can write your report on this notebook. The report must include visualization of your results. Train your model at least with 2 different sets of hyperparameters and in visualization section compare their output.


### Good Luck!